In [1]:
import pandas as pd
from pandas import read_csv
from pandas.api.types import is_numeric_dtype
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score
import xgboost as xgb

dataset = read_csv('/Users/Cristy/Downloads/task1/X_train.csv')
dataset.drop('id',axis=1,inplace=True)
y_train=read_csv('/Users/Cristy/Downloads/task1/y_train.csv')
dataset['y']=y_train.y

#Fill-in with median
df2=dataset.fillna(dataset.median())

# Outliers:

## Isolation Forest 

In [2]:
clf = IsolationForest(max_samples=100, random_state=42, behaviour="new",contamination=.1)

clf.fit(df2.drop('y', axis = 1))
y_pred_train = clf.predict(df2.drop('y', axis = 1))
df2_clean = df2[np.where(y_pred_train == 1, True, False)]

New dataset without outliers:

In [3]:
df2_clean.shape

(1090, 833)

In [4]:

features= np.array(df2_clean.drop('y', axis = 1))
labels=np.array(df2_clean['y'])
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf=rf.fit(features,labels)
rf.feature_importances_
sfm=SelectFromModel(rf,prefit=True)
features_new=sfm.transform(features)

features_new.shape

(1090, 95)

### Selecting more features (threshold greater than mean of feature importance values)

In [5]:
sfm3=SelectFromModel(rf,prefit=True,threshold=1e-3)
features_new3=sfm3.transform(features)
features_new3.shape

(1090, 120)

In [6]:
train_features, test_features, train_labels, test_labels = train_test_split(features_new3, labels, test_size = 0.25, random_state = 42)


In [7]:
model = xgb.XGBRegressor(learning_rate = 0.1, max_depth = 3)
model.fit(train_features, train_labels)
y_train_pred = model.predict(test_features)
score = r2_score(test_labels, y_train_pred)
print('train score: ', score)

[17:00:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train score:  0.5585837672002434


In [8]:
test = read_csv('/Users/Cristy/Downloads/task1/X_test.csv')
test.drop('id',axis=1,inplace=True)
df2=test.fillna(dataset.median())
test_features=np.array(df2)
test_features_new=sfm3.transform(test_features)
predictions = model.predict(test_features_new)
out = pd.DataFrame({'id':[float(i) for i in range(0,len(test_features_new))],'y':predictions})
out.to_csv("sampleIsolXGB2.csv",index=False)